In [1]:
# %%capture
# If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed

!pip install --no-index /kaggle/input/packages/wheelhouse/bitsandbytes-0.43.0-py3-none-manylinux_2_24_x86_64.whl --find-links=/kaggle/input/packages/wheelhouse/
!pip install --no-index /kaggle/input/packages/wheelhouse/accelerate-0.28.0-py3-none-any.whl --find-links=/kaggle/input/packages/wheelhouse/
!pip install --no-index /kaggle/input/packages/wheelhouse/transformers-4.39.0-py3-none-any.whl --find-links=/kaggle/input/packages/wheelhouse/
!pip install --no-index /kaggle/input/packages/wheelhouse/optimum-1.17.1-py3-none-any.whl --find-links=/kaggle/input/packages/wheelhouse/
!pip install --no-index /kaggle/input/packages/wheelhouse/peft-0.9.0-py3-none-any.whl --find-links=/kaggle/input/packages/wheelhouse/
!pip install --no-index /kaggle/input/packages/wheelhouse/trl-0.8.0-py3-none-any.whl --find-links=/kaggle/input/packages/wheelhouse/
!pip install --no-index /kaggle/input/packages/wheelhouse/datasets-2.18.0-py3-none-any.whl --find-links=/kaggle/input/packages/wheelhouse/

Looking in links: /kaggle/input/packages/wheelhouse/
Processing /kaggle/input/packages/wheelhouse/bitsandbytes-0.43.0-py3-none-manylinux_2_24_x86_64.whl
Looking in links: /kaggle/input/packages/wheelhouse/
Processing /kaggle/input/packages/wheelhouse/accelerate-0.28.0-py3-none-any.whl
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2
Looking in links: /kaggle/input/packages/wheelhouse/
Processing /kaggle/input/packages/wheelhouse/transformers-4.39.0-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1
Looking in links: /kaggle/input/packages/wheelhouse/
Processing /kaggle/input/packages/wheelhouse/optimum-1.17.1-py3-none-any.whl
Processing /kaggle/input/packages/wheelhouse/coloredlogs-15.0.1-py2.py3-none-any.whl (from optimum=

In [2]:
!wandb disabled

W&B disabled.


In [3]:
from accelerate.utils import BnbQuantizationConfig
from accelerate import Accelerator
import transformers
from transformers import TrainingArguments
import optimum
import bitsandbytes
from peft import prepare_model_for_kbit_training, LoftQConfig, LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from datasets import Dataset

2024-03-23 22:37:54.535808: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 22:37:54.535920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 22:37:54.692958: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import datetime
start_time = datetime.datetime.now()

In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

accelerator = Accelerator()

# Comment/Uncomment and use as per wish

MODEL_PATH = "/kaggle/input/gemma/transformers/7b-it/2"
# MODEL_PATH = "/kaggle/input/gemma/transformers/2b-it/2"
# MODEL_PATH = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
# MODEL_PATH = "/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1"
# MODEL_PATH = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"
# MODEL_PATH = "/kaggle/input/llama-2/pytorch/13b-chat-hf/1"

# Found a good blog to catch me up fast!
# https://huggingface.co/blog/4bit-transformers-bitsandbytes
# https://huggingface.co/docs/transformers/v4.38.1/en/quantization#compute-data-type
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)


model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config = quantization_config,
)


# model = model.to_bettertransformer()
#model = accelerator.prepare(model)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
import pandas as pd
from tqdm import tqdm

DEBUG = False

# TEST_DF_FILE = '/kaggle/input/llm-prompt-recovery/test.csv'
TEST_DF_FILE = '/kaggle/input/llm-prompt-recovery/test.csv'
SUB_DF_FILE = '/kaggle/input/llm-prompt-recovery/sample_submission.csv'
NROWS = None if DEBUG else None

if DEBUG:
    TEST_DF_FILE = '/kaggle/input/gemma-rewrite-nbroad/nbroad-v2.csv'
    SUB_DF_FILE = TEST_DF_FILE

tdf = pd.read_csv(TEST_DF_FILE, nrows=NROWS, usecols=['id', 'original_text', 'rewritten_text'])
sub = pd.read_csv(SUB_DF_FILE, nrows=NROWS, usecols=['id', 'rewrite_prompt'])

In [8]:
def truncate_txt(text, length):
    text_list = text.split()
    
    if len(text_list) <= length:
        return text
    
    return " ".join(text_list[:length])


def gen_prompt(og_text, rewritten_text):
    
    # Truncate the texts to first 200 words for now
    # As we are having memory issues on Mixtral8x7b
    og_text = truncate_txt(og_text, 200)
    rewritten_text = truncate_txt(rewritten_text, 200)
    
    return f"""Original Text:\"""{og_text}\"""\nRewritten Text:\"""{rewritten_text}\"""\nGiven are 2 texts, the Rewritten text was created from the Original text using the google Gemma model. You are trying to understand how the original text was transformed into a new version. Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten text. Start directly with the prompt, that's all I need. Output should be one line ONLY."""

# QLoRA Fine-tuning

In [9]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    target_modules = "all-linear",
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    
)
model = get_peft_model(model, peft_config)

In [10]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

In [11]:
# `LLM Prompt Recovery - Synthetic Datastore dataset` by @dschettler8845
df1 = pd.read_csv("/kaggle/input/llm-prompt-recovery-synthetic-datastore/gemma1000_w7b.csv")
df1 = df1[["original_text", "rewrite_prompt", "gemma_7b_rewritten_text_temp0"]]
df1 = df1.rename(columns={"gemma_7b_rewritten_text_temp0":"rewritten_text"})
df1.head(2)

# `3000 Rewritten texts - Prompt recovery Challenge` by @dipamc77
df2 = pd.read_csv("/kaggle/input/3000-rewritten-texts-prompt-recovery-challenge/prompts_0_500_wiki_first_para_3000.csv")
df2.head(2)

# Merge all datasets
df = pd.concat([df1, df2], axis=0)
df = df.sample(4000).reset_index(drop=True) # to reduce training time we are only using 2k samples
df.head(5)

,original_text,rewrite_prompt,rewritten_text
0,Andrius Šidlauskas may refer to:\n Andrius Šid...,Change this into an intellectual property righ...,"Sure, here's the intellectual property rights ..."
1,"Georgina Klitgaard ( Berrian; July 3, 1889/189...",Make it a guide to deciphering ancient runes.,The text does not mention runes or decoding th...
2,The black-collared lovebird (Agapornis swinder...,Recast it as a travel blog post from an exotic...,## A Flight Through the Fabled Fig Trees of Za...
3,Karl Theodor Leopold Liebe (11 February 1828 -...,Transform this into a fairy tale.,In a quaint town nestled amidst emerald meadow...
4,Gökçe Fırat Çulhaoğlu (born 8 March 1974) is t...,Transform this into a fairy tale.,"In a land shrouded in whispers of the north, a..."


In [12]:
dict = {
    "id": [str(x) for x in range(0,len(df))],
    "original_text": [str(x) for x in df.original_text],
    "rewrite_prompt": [str(x) for x in df.rewrite_prompt],
    "rewritten_text": [str(x) for x in df.rewritten_text],
}
dict['rewritten_text'][6]

"A heartfelt public apology:\n\nI deeply regret that I inadvertently omitted the information about the renowned Latvian artist, Zanis Waldheims, in my previous text. The omission was a result of a lapse of memory and not intentional.\n\nWaldheims' artistic contributions and the impact he had on the field of geometric abstraction are undeniable. It is a matter of great regret that I overlooked his impactful work and the cultural heritage he left behind.\n\nI extend my sincerest apologies to the art community and all those who have been affected by my oversight. I understand the importance of acknowledging and celebrating the achievements of individuals who have made a positive contribution to the world, and I am committed to recti"

In [13]:
ds = Dataset.from_dict(dict)
ds[0].keys()

dict_keys(['id', 'original_text', 'rewrite_prompt', 'rewritten_text'])

In [14]:
def truncate_text(text, max_len): #without this    
    text_split = text.split()    
    if len(text_split)<=max_len:
        return text
    else:
        return " ".join(text_split[:max_len])

def tokenize(example):
    for ot, rt, rp in zip(example["original_text"], example["rewritten_text"], example["rewrite_prompt"]):
        original_text = truncate_text(ot, 200)
        rewritten_text = truncate_text(rt, 200)
        rewrite_prompt = truncate_text(rp, 100)
    
        template = f"""Original Text:\"""{original_text}\"""\nRewritten Text:\"""{rewritten_text}\"""\nGiven are 2 texts, the Rewritten text was created from the Original text using the google Gemma model. You are trying to understand how the original text was transformed into a new version. Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten text. Start directly with the prompt, that's all I need. Output should be one line ONLY.\n\nResponse:\n\"{rewrite_prompt}\""""
        tkn = tokenizer(template, padding=True)
    return {**tkn}

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
ds_tokenize = ds.map(tokenize)
ds_tokenize[0].keys()

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

dict_keys(['id', 'original_text', 'rewrite_prompt', 'rewritten_text', 'input_ids', 'attention_mask'])

In [15]:
ds_split = ds_tokenize.train_test_split(test_size=0.20)

train_ds = ds_split['train']
test_ds = ds_split['test']

In [16]:
from transformers import Trainer, DataCollatorForLanguageModeling
trainer = Trainer(
    model=model, # lora enabled
    train_dataset=ds_tokenize,    
    args=TrainingArguments(
        per_device_train_batch_size=1,
        learning_rate=2e-5,
        fp16=True,
        output_dir=".",
        optim="paged_adamw_8bit",
        num_train_epochs=1
    ),
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    #compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [17]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
500,0.713200
1000,0.066200
1500,0.067000
2000,0.063100
2500,0.064100
3000,0.063000
3500,0.064800
4000,0.062600


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /kaggle/input/gemma/transformers/7b-it/2 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /kaggle/input/gemma/transformers/7b-it/2 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch

TrainOutput(global_step=4000, training_loss=0.14550713539123536, metrics={'train_runtime': 7693.016, 'train_samples_per_second': 0.52, 'train_steps_per_second': 0.52, 'total_flos': 2.1315130391402496e+16, 'train_loss': 0.14550713539123536, 'epoch': 1.0})

In [18]:
trainer.model.save_pretrained("My fine tuned model")

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /kaggle/input/gemma/transformers/7b-it/2 - will assume that the vocabulary was not modified.
  warnings.warn(


# Prediction

In [19]:
# TEST_DF_FILE = '/kaggle/input/testsmall/smalltest.csv'
TEST_DF_FILE = '/kaggle/input/llm-prompt-recovery/test.csv'
SUB_DF_FILE = '/kaggle/input/llm-prompt-recovery/sample_submission.csv'
NROWS = None if DEBUG else None

if DEBUG:
    TEST_DF_FILE = '/kaggle/input/gemma-rewrite-nbroad/nbroad-v2.csv'
    SUB_DF_FILE = TEST_DF_FILE

tdf = pd.read_csv(TEST_DF_FILE, nrows=NROWS, usecols=['id', 'original_text', 'rewritten_text'])

In [20]:
import gc
import re

device = accelerator.device
tdf['id'] = sub['id'].copy()

pbar = tqdm(total=tdf.shape[0])

it = iter(tdf.iterrows())
idx, row = next(it, (None, None))

# https://www.kaggle.com/competitions/llm-prompt-recovery/discussion/481116
DEFAULT_TEXT = "Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style."

res = []

while idx is not None:
    
    if (datetime.datetime.now() - start_time) > datetime.timedelta(hours=8, minutes=30):
        res.append([row["id"], DEFAULT_TEXT])
        idx, row = next(it, (None, None))
        pbar.update(1)
        continue
        
    torch.cuda.empty_cache()
    gc.collect()
        
    try:        
        messages = [
            {
                "role": "user",
                "content": gen_prompt(row["original_text"], row["rewritten_text"])
            }
        ]
        encoded_input = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(device)
        
        with torch.no_grad():
            encoded_output = model.generate(encoded_input, max_new_tokens=50, do_sample=True, pad_token_id=tokenizer.eos_token_id)
        
        decoded_output = tokenizer.batch_decode(encoded_output, skip_special_tokens=True)[0]
        decoded_output = result = re.sub(r"[\s\S]*\[\/INST\]", '', decoded_output, 1)
                
        res.append([row["id"], decoded_output])
                            
    except Exception as e:
        print(f"ERROR: {e}")
        res.append([row["id"], DEFAULT_TEXT])
        
    finally:
        idx, row = next(it, (None, None))
        pbar.update(1)

        
pbar.close()

  0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 1/1 [00:24<00:00, 24.10s/it]


In [21]:
# sub["rewrite_prompt"] = tdf['rewrite_prompt'].copy()
# sub.to_csv("submission.csv", index=False)
sub = pd.DataFrame(res, columns=['id', 'rewrite_prompt'])

sub.to_csv("sample_submission.csv", index=False)
sub.to_csv("submission.csv", index=False)

In [22]:
sub

,id,rewrite_prompt
0,9559194,"user\nOriginal Text:""""""The competition dataset..."


In [23]:
res

[[9559194,
  'user\nOriginal Text:"""The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set."""\nRewritten Text:"""Here is your shanty: (Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they\'ve been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we\'ll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be"""\nGiven are 2 texts, the Rewritten text was created from the Orig